In [1]:
import pandas as pd
import json
import numpy as np
import datetime
from pymongo import MongoClient
import Orange
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Create Mongo client instance
mongo_client=MongoClient()

In [16]:
import talib

In [2]:
dbeq=mongo_client['NSEEQ']
data=dbeq['ACC'].find()
eqdf=pd.DataFrame(list(data))
eqdf.drop('_id',inplace=True,axis=1)
eqdf.drop_duplicates(inplace=True)


In [3]:
df=eqdf.copy()
df['20dhigh']=df['High'].rolling(20).max()
df['20dlow']=df['Low'].rolling(20).min()
df['20davg']=df['Close'].rolling(20).mean()
df['20dhlavg']=((df['High']+df['Low'])/2).rolling(20).mean()
df.fillna(0,inplace=True)

In [4]:
df.set_index('Datetime',inplace=True)

In [5]:
# import mitosheet
# mitosheet.sheet(df, view_df=True)

In [6]:
df

,Pclose,Open,High,Low,Close,Vwap,Volume,#Trades,Delivery,PerDelivery,20dhigh,20dlow,20davg,20dhlavg
Datetime,,,,,,,,,,,,,,
2019-10-01,1608.15,1611.65,1627.40,1555.05,1575.70,1587.87,460449,28156,49503,10.75,0.00,0.0,0.0000,0.00000
2019-10-03,1575.70,1564.00,1582.85,1537.00,1545.75,1556.06,715946,32528,277689,38.79,0.00,0.0,0.0000,0.00000
2019-10-04,1545.75,1558.10,1559.30,1482.55,1488.10,1514.83,620192,29156,154647,24.94,0.00,0.0,0.0000,0.00000
2019-10-07,1488.10,1496.00,1496.15,1430.15,1435.10,1457.53,1135461,55631,465421,40.99,0.00,0.0,0.0000,0.00000
2019-10-09,1435.10,1430.00,1473.95,1416.45,1467.10,1445.49,1260259,51936,353890,28.08,0.00,0.0,0.0000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,2266.45,2266.00,2274.00,2207.50,2215.05,2233.04,290265,20358,114997,39.62,2445.00,2205.0,2300.4400,2307.23125
2021-12-20,2215.05,2194.90,2199.65,2087.80,2115.90,2124.75,307702,18217,114869,37.33,2428.25,2087.8,2287.5575,2294.41750
2021-12-21,2115.90,2136.00,2179.00,2123.85,2138.55,2150.71,294036,12377,147882,50.29,2428.25,2087.8,2274.2900,2282.73875


In [7]:
df=df.loc[df['20dhigh']>0]
df['20highdiff']=np.log(df['20dhigh']-df['20dhlavg'])
df['20lowdiff']=np.log(df['20dhlavg']-df['20dlow'])
df['str']=df['20highdiff']+df['20lowdiff']

In [8]:
def my_rank(x):
   return pd.Series(x).rank(pct=True).iloc[-1]

In [9]:
df['STR_Rank']=(df['str'].rolling(100).apply(my_rank))*100

In [10]:
df.fillna(0,inplace=True)

In [11]:
df

,Pclose,Open,High,Low,Close,Vwap,Volume,#Trades,Delivery,PerDelivery,20dhigh,20dlow,20davg,20dhlavg,20highdiff,20lowdiff,str,STR_Rank
Datetime,,,,,,,,,,,,,,,,,,
2019-10-31,1532.25,1541.70,1574.20,1536.25,1568.05,1555.75,712719,19911,200164,28.08,1627.40,1416.45,1510.7325,1513.85000,4.732243,4.578826,9.311069,0.0
2019-11-01,1568.05,1573.00,1575.60,1528.00,1542.20,1553.29,887153,24635,219808,24.78,1582.85,1416.45,1509.0575,1511.87875,4.262275,4.558380,8.820655,0.0
2019-11-04,1542.20,1549.80,1590.50,1542.20,1564.65,1575.06,1094603,35198,346958,31.70,1590.50,1416.45,1510.0025,1512.20000,4.360548,4.561741,8.922288,0.0
2019-11-05,1564.65,1561.00,1564.00,1510.40,1514.65,1531.80,914025,44378,296631,32.45,1590.50,1416.45,1511.3300,1513.01375,4.350101,4.570203,8.920304,0.0
2019-11-06,1514.65,1520.00,1531.20,1501.85,1517.40,1516.62,885965,30581,248810,28.08,1590.50,1416.45,1515.4450,1515.68250,4.315052,4.597466,8.912517,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,2266.45,2266.00,2274.00,2207.50,2215.05,2233.04,290265,20358,114997,39.62,2445.00,2205.00,2300.4400,2307.23125,4.925577,4.627237,9.552814,23.0
2021-12-20,2215.05,2194.90,2199.65,2087.80,2115.90,2124.75,307702,18217,114869,37.33,2428.25,2087.80,2287.5575,2294.41750,4.896589,5.330869,10.227458,60.0
2021-12-21,2115.90,2136.00,2179.00,2123.85,2138.55,2150.71,294036,12377,147882,50.29,2428.25,2087.80,2274.2900,2282.73875,4.980253,5.272685,10.252939,61.0


In [12]:
### This whole section to plot charts in Dash
def genrate_graphs():
    app = JupyterDash(__name__)
    app.layout = html.Div(children=[
            html.H1(children='Hello Dash'),

            html.Div(children='''
                Dash: A web application framework for your data.
            '''),
            dcc.Checklist(
                id='toggle-rangeslider',
                options=[{'label': 'Include Rangeslider', 
                          'value': 'slider'}],
                value=['slider']
            ),
            dcc.Graph(
                id='graph',
                # figure=fig
            )
        ])
    @app.callback(Output("graph", "figure"), [Input("toggle-rangeslider", "value")])
    def show_chart(value):
        fig=make_subplots(rows=2,cols=1,column_widths=[1.0],
                                  row_heights=[1.0,0.3],shared_xaxes=True,vertical_spacing=0.01,
                                  specs=[
                                      [{"type":"xy","secondary_y": True}],#None],
                                      # [{"type":"Candlestick"}],#None],                              
                                      # [{"type":"xy","secondary_y": True}],#None],
                                      [{"type":"xy"}],#None]])
                                        ]
                                 )
        fig.add_trace(
                    go.Candlestick(name='price',x=df.index,
                                open=df['Open'],
                                high=df['High'],
                                low=df['Low'],
                                close=df['Close']),row=1,col=1,secondary_y=False)
        fig.update_layout(xaxis_rangeslider_visible=False)
       
        fig.add_trace(go.Scatter(x=df.index,y=df['20davg'], name="20 Days Avg",mode='lines'),
                              row=1,col=1)
        fig.add_trace(go.Scatter(x=df.index,y=df['STR_Rank'], name="str",mode='lines'),
                              row=2,col=1)
        fig.update_xaxes(row=1,col=1,
                         rangeselector=dict(buttons=list(
                             [dict(count=1, label="1m", step="month", stepmode="backward"),
                              dict(count=6, label="6m", step="month", stepmode="backward"),
                              dict(count=1, label="YTD", step="year", stepmode="todate"),
                              dict(count=1, label="1y", step="year", stepmode="backward"),
                              dict(step="all")
                             ])
                                           ),
                         rangebreaks=[
                             dict(bounds=["sat", "mon"])#hide weekends
                         ]
                        )
                # rangebreaks=[
            #     dict(bounds=[17, 9], pattern="hour"), #hide hours outside of 9am-5pm
            # ]
        return fig
        
    app.run_server(mode='jupyterlab',port = 8096)


In [13]:
genrate_graphs()

In [14]:
np.log(-1)

nan

C:\Users\admin\AppData\Local\Temp/ipykernel_2992/153577755.py:1: RuntimeWarning: invalid value encountered in log
  np.log(-1)
